## Prompt Injection

### Mock data maken

In [1]:
import pandas as pd
from faker import Faker
import random
import json
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import os 

#!pip install transformers datasets
#!pip install faker
#!pip install torch
#!pip install tensorflow
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!pip install sentencepiece
#!pip install tf-keras
#!pip install huggingface_hub
#!pip install python-dotenv

In [2]:
fake = Faker()
Faker.seed(42)
random.seed(42)

In [3]:
def generate_patient_data(num_patients=100):
    patient_data = []
    for _ in range(num_patients):
        patient = {
            "PatiëntNaam": fake.name(),
            "Leeftijd": random.randint(0, 100),
            "Geslacht": random.choice(["Man", "Vrouw"]),
            "Diagnose": random.choice(["Hypertensie", "Diabetes Type 2", "Astma", "Hartfalen", "Depressie"]),
            "Medicatie": random.choice(["Metformine", "Paracetamol", "Lisinopril", "Salbutamol", "Sertraline"]),
            "OpnameDatum": fake.date_this_year().strftime("%Y-%m-%d"),
            "OntslagDatum": fake.date_this_year().strftime("%Y-%m-%d"),
            "Afdeling": random.choice(["Cardiologie", "Psychiatrie", "Endocrinologie", "Longgeneeskunde"]),
        }
        patient_data.append(patient)
    return pd.DataFrame(patient_data)

In [4]:
patient_df = generate_patient_data(100)
print(patient_df.head())

       PatiëntNaam  Leeftijd Geslacht     Diagnose   Medicatie OpnameDatum  \
0     Allison Hill        81      Man  Hypertensie  Lisinopril  2024-03-27   
1    Megan Mcclain        28      Man  Hypertensie  Sertraline  2024-02-03   
2   Allen Robinson        75    Vrouw  Hypertensie  Metformine  2024-07-15   
3  Cristian Santos        27      Man    Depressie  Sertraline  2024-07-30   
4    Kevin Pacheco        71      Man    Depressie  Salbutamol  2024-01-03   

  OntslagDatum     Afdeling  
0   2024-02-24  Psychiatrie  
1   2024-08-17  Cardiologie  
2   2024-08-21  Cardiologie  
3   2024-06-11  Cardiologie  
4   2024-10-21  Psychiatrie  


### Mock data omzetten naar ander format

Llama werkt het best met een dataset dat bestaat uit teksten in plaats van rijen en kolommen.

In [5]:
def convert_to_llama_format(df):
    data = []
    for _, row in df.iterrows():
        example = {
            "instruction": "Geef de diagnose van deze patiënt.",
            "input": f"Naam: {row['PatiëntNaam']}, Leeftijd: {row['Leeftijd']}, Geslacht: {row['Geslacht']}, Medicatie: {row['Medicatie']}, OpnameDatum: {row['OpnameDatum']}, Afdeling: {row['Afdeling']}",
            "output": row["Diagnose"]
        }
        data.append(example)
    return data

In [6]:
llama_data = convert_to_llama_format(patient_df)

In [7]:
output_file = "mock_patient_data.jsonl"
with open(output_file, "w") as f:
    for entry in llama_data:
        f.write(json.dumps(entry) + "\n")

print(f"Dataset opgeslagen in {output_file}")

Dataset opgeslagen in mock_patient_data.jsonl


## LLaMA toepassen

Om Llama te gebruiken via Huggingface moet er toegang geven worden aan je huggingface account. Daarom moet er ingelogd worden op huggingface


In [8]:
load_dotenv("HF_TOKEN.env")
hf_token = os.getenv("HF_TOKEN")
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [19]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

model.safetensors:   9%|9         | 231M/2.47G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Laad de JSONL-bestand in een dataset
dataset = load_dataset("json", data_files={"train": "mock_patient_data.jsonl"}, split="train")

# Tokeniseer de dataset
def tokenize_function(examples):
    return tokenizer(examples["input"], examples["output"], truncation=True, padding="max_length", max_length=512)

# Pas de tokenisatie toe
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
# Stel de trainingparameters in
training_args = TrainingArguments(
    output_dir="./llama_finetuned",  # Waar de getrainde modelbestanden worden opgeslagen
    evaluation_strategy="epoch",  # Evaluatie per epoch
    learning_rate=2e-5,  # Standaard learning rate voor fine-tuning
    per_device_train_batch_size=4,  # Batchgrootte per GPU
    num_train_epochs=3,  # Aantal epochs
    weight_decay=0.01,  # Gewicht-decay voor regularisatie
)

# Maak een Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Start de training
trainer.train()

In [ ]:
# Voorbeeldinvoer
patient_input = "Naam: John Doe, Leeftijd: 45, Geslacht: Man, Medicatie: Lisinopril, OpnameDatum: 2024-04-10, Afdeling: Cardiologie"

# Tokeniseer de invoer
inputs = tokenizer(patient_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Voorspel de output
outputs = model.generate(inputs["input_ids"])

# Decodeer de output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Voorspelde diagnose: {generated_text}")


In [ ]:
# Voorbeeldinvoer
patient_input = "Naam: John Doe, Leeftijd: 45, Geslacht: Man, Medicatie: Lisinopril, OpnameDatum: 2024-04-10, Afdeling: Cardiologie"

# Tokeniseer de invoer
inputs = tokenizer(patient_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Voorspel de output
outputs = model.generate(inputs["input_ids"])

# Decodeer de output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Voorspelde diagnose: {generated_text}")